### LABORATORIO 4


In [1]:
# LIBRERIAS
import os #Para mejor navegacion en directorios
import numpy as np #Para hacer cálculos matemáticos
from matplotlib import pyplot #Para hacer gráficos
%matplotlib inline
from scipy import optimize #Para entrenar nuestros optimizadores
import pandas as pd #Para leer datasets
from sklearn.preprocessing import LabelEncoder #Para convertir palabras en números o valores maquina

In [3]:

import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split

CSV_PATH = "/content/covtype.csv"
target_col = "Cover_Type"

df = pd.read_csv(CSV_PATH)
print("Forma original:", df.shape)

# 20k
N = 20_000
if len(df) > N:
    df_20k = (df.groupby(target_col, group_keys=False)
                .apply(lambda g: g.sample(n=min(len(g), max(1, int(len(g) * N / len(df)))), random_state=42))
                .reset_index(drop=True))
    if len(df_20k) > N:
        df_20k = df_20k.sample(n=N, random_state=42)
else:
    df_20k = df.copy()

print("Forma 20k:", df_20k.shape)
df_20k.to_csv("/content/covtype_20k.csv", index=False)

# Definir X, y
LABELS = np.sort(df_20k[target_col].unique())
y = pd.Categorical(df_20k[target_col], categories=LABELS, ordered=True).codes
num_labels = len(LABELS)
print("Clases originales:", LABELS)
print("Clases codificadas (0..K-1):", sorted(np.unique(y)))

X = df_20k.drop(columns=[target_col]).to_numpy(dtype=float)
mu = X.mean(axis=0)
sigma = X.std(axis=0, ddof=0)
sigma[sigma == 0] = 1.0
X = (X - mu) / sigma

# Split estándar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Forma:", X_train.shape, X_test.shape)


Forma original: (81048, 55)
Forma 20k: (19998, 55)


/tmp/ipython-input-258965537.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(len(g), max(1, int(len(g) * N / len(df)))), random_state=42))


Clases originales: [1. 2. 3. 4. 5. 6. 7.]
Clases codificadas (0..K-1): [np.int8(0), np.int8(1), np.int8(2), np.int8(3), np.int8(4), np.int8(5), np.int8(6)]
Forma: (15998, 54) (4000, 54)


In [4]:
# Imprime la primera fila de características de la primera persona del dataset
print(X[0,:])

# Imprime el vector de etiquetas (y), que indica a qué grupo pertenece cada persona en el dataset
print(y)

[ 1.16204139 -1.166078    0.58137729  2.99345097 -1.36312617  0.47818423
 -0.63748858 -1.14396312 -0.15289832  0.08034289  0.40609041 -0.08182416
 -0.28935484 -0.24735266 -0.06761107 -0.08516423 -0.1078655  -0.10101525
 -0.04363264 -0.09204353  0.         -0.01732397 -0.01224898 -0.16387478
 -0.07053456 -0.52655121 -0.07089168 -0.04587625  0.         -0.14521559
 -0.09037128 -0.14841456 -0.08120252 -0.23147332 -0.01414426 -0.07124705
 -0.23963276 -0.15224087  0.         -0.02450348 -0.01581416 -0.0070716
  1.3362727  -0.3296354  -0.06416613 -0.09422857 -0.08780519 -0.0070716
 -0.04065578 -0.01414426 -0.0187125  -0.10276878 -0.10276878 -0.09583578]
[0 0 0 ... 6 6 6]


In [10]:
# Funcion normaliza datos
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [11]:
# LLAMAR A LA FUNCIÓN NORMALIZADORA
X_norm, mu, sigma = featureNormalize(X)

In [12]:
print(X_norm[0,:])
print(y)

[ 1.16204139 -1.166078    0.58137729  2.99345097 -1.36312617  0.47818423
 -0.63748858 -1.14396312 -0.15289832  0.08034289  0.40609041 -0.08182416
 -0.28935484 -0.24735266 -0.06761107 -0.08516423 -0.1078655  -0.10101525
 -0.04363264 -0.09204353         nan -0.01732397 -0.01224898 -0.16387478
 -0.07053456 -0.52655121 -0.07089168 -0.04587625         nan -0.14521559
 -0.09037128 -0.14841456 -0.08120252 -0.23147332 -0.01414426 -0.07124705
 -0.23963276 -0.15224087         nan -0.02450348 -0.01581416 -0.0070716
  1.3362727  -0.3296354  -0.06416613 -0.09422857 -0.08780519 -0.0070716
 -0.04065578 -0.01414426 -0.0187125  -0.10276878 -0.10276878 -0.09583578]
[0 0 0 ... 6 6 6]


In [14]:
# PREPARAR LOS DATOS PARA ENTRENAR CON DATOS NORMALIZADOS
m, n = X.shape
X = X_norm

In [15]:
#FUNCIÓN SIGMOIDE
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [16]:

def calcularCosto(theta, X, y):
    m = y.size

    J = 0
    h = sigmoid(X.dot(theta.T))

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    return J

In [17]:
# ENTRENAMIENTO
def descensoGradiente(theta, X, y, alpha, num_iters):
    m = y.shape[0]
    theta = theta.copy()

    J_history = []

    # Repetir
    for i in range(num_iters):
        h = sigmoid(X.dot(theta.T))

        # Fórmula para actualizar
        theta = theta - (alpha / m) * (h - y).dot(X)

        # Guardar costo de aprendizaje
        J_history.append(calcularCosto(theta, X, y))

    return theta, J_history


In [18]:
# FUNCIÓN NUEVA DE ENTRENAMIENTO (Con regularización)

def lrCostFunction(theta, X, y, lambda_):

    m = y.size

    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad


<h1 align="center">FUNCION ONE VS ALL</h1>

### Descripción
El método **One vs All (OvA)** extiende la regresión logística binaria a problemas multiclase.  
Se entrena un modelo por cada clase: cada modelo predice si un ejemplo pertenece o no a esa clase.  
La clase final se elige tomando el modelo con la probabilidad más alta.

---

### Fórmulas

**Hipótesis logística**

hθ(x) = 1 / (1 + e^(-θᵀx))

**Función de costo regularizada**

J(θ) = -(1/m) Σ [ y(i) log(hθ(x(i))) + (1 - y(i)) log(1 - hθ(x(i))) ]   
       + (λ / 2m) Σ θⱼ²   (para j ≥ 1)

**Actualización por descenso de gradiente**

θ := θ - α (1/m) Σ ( hθ(x(i)) - y(i) ) x(i)

---

### Procedimiento clásico

1. Para cada clase c en el conjunto de etiquetas:  
   - Convertir las etiquetas en binarias (1 si es la clase c, 0 en otro caso).  
   - Entrenar un modelo de regresión logística mediante descenso de gradiente.  
   - Guardar los parámetros obtenidos en θc.  

2. Para clasificar un nuevo ejemplo:  
   - Calcular la probabilidad de pertenecer a cada clase.  
   - Seleccionar la clase con la probabilidad más alta.  


In [20]:
# (One vs All)
def OneVsAll(X, y, num_labels, lambda_):
  alpha = 0.001
  num_iters = 100000

  m, n = X.shape
  all_theta = np.zeros((num_labels, n + 1))

  X = np.concatenate([np.ones((m, 1)), X], axis=1)

  for c in np.arange(num_labels):
      initial_theta = np.zeros(n + 1)

      y_actual = np.where(y == c, 1, 0)

      theta, J_history = descensoGradiente(initial_theta, X, y_actual, alpha, num_iters)

      all_theta[c] = theta

      pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
      pyplot.xlabel('Numero de iteraciones')
      pyplot.ylabel('Costo J')

  return all_theta

<h1 align="center">FUNCION ONE VS ALL OPTIMIZADO</h1>

Esta versión implementa un optimizador avanzado (`scipy.optimize.minimize`) en lugar del descenso de gradiente manual.  
Los parámetros θ convergen más rápido y de forma más estable.

## Hipótesis logística

$$
h_\theta(x) = \frac{1}{1 + e^{-\theta^T x}}
$$

## Función de costo regularizada

$$
J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}
\Big[ y^{(i)} \log(h_\theta(x^{(i)})) +
(1 - y^{(i)}) \log(1 - h_\theta(x^{(i)})) \Big]
+ \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2
$$

## Ventajas del método optimizado
- No requiere tasa de aprendizaje α  
- Usa algoritmos de optimización eficientes (ej. gradiente conjugado)  
- Converge en menos iteraciones
  


In [21]:
# (One vs All Optimizado)
def OneVsAllOM(X, y, num_labels, lambda_):

    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 10}


        res = optimize.minimize(lrCostFunction,
            initial_theta,
            (X, (y == c), lambda_),
            jac=True,
            method='CG',
            options=options)

        all_theta[c] = res.x

    return all_theta

In [22]:
# ¡ENTRENAR!
lambda_ = 0.1

num_labels = len(LABELS)

all_theta = OneVsAllOM(X, y, num_labels, lambda_)
print(f"¡Modelo entrenado! Forma: {all_theta.shape}")


¡Modelo entrenado! Forma: (7, 55)


In [23]:
print(all_theta)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [24]:
#ENTRENA AL MODELO ONE VS ALL CLASICO

def OneVsAll(X, y, num_labels, lambda_):
    alpha = 0.01
    num_iters = 1000
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        y_actual = np.where(y == c, 1, 0)
        theta, J_history = descensoGradiente(initial_theta, X, y_actual, alpha, num_iters)
        all_theta[c] = theta

    return all_theta

all_theta_clasico = OneVsAll(X, y, num_labels, lambda_)
print(f"¡Modelo clásico entrenado! Forma: {all_theta_clasico.shape}")

¡Modelo clásico entrenado! Forma: (7, 55)


In [25]:
print(all_theta_clasico)

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan 

<a id="section3"></a>
#### 1.4.1 Prediccion One-vs-all

In [26]:
# PREDICCIONES
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p


print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[100:145, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((45, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

print(y[100:145])

(19998, 54)
Precision del conjuto de entrenamiento: 23.27%
(45, 54)
(45, 55)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
